<a href="https://colab.research.google.com/github/cr21/Unsupervised-Machine-Learning-Clustering/blob/main/HarrFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Dataset/

/content/drive/MyDrive/Dataset


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection  import mutual_info_classif
from sklearn import linear_model
from tqdm import tqdm

#https://www.ccs.neu.edu/home/vip/teach/DMcourse/3_dim_reduction/notes_slides/viola-cvpr-01.pdf


In [2]:
# [[15, 25], [2, 14]]
def getArea(p1, p2):
  A = p1 # 15 25
  B = p2 # 2 14
  C = [p1[0], p2[1]]  # 15  14
  D = [p2[0], p1[1]] # 2  25
  # 2 15  14  25
  # 13 * 11 
  return ((p2[0] - p1[0]) * (p2[1] - p1[1]))  

In [15]:
def HalfArea(p1, p2, axis = 0):
  if axis == 0  :
    return [(p1[0]+p2[0] )// 2, p1[1]]
  else:
    return [p1[0], ((p1[1] + p2[1]) // 2)]



In [16]:
def generateRandomRectangles(num=100) :
  count = 1
  rects = []
  while count <= 100:
    top =[]
    bottom =[]

    top.append(np.random.randint(27))
    top.append(np.random.randint(27))
    bottom.append(np.random.randint(27))
    bottom.append(np.random.randint(27))

    rect = [top, bottom]

    if rect in rects :
      continue

    # validate the rectange

    if (top[0] < bottom[0]) and (top[1] < bottom[1]) :
      area = getArea(top, bottom)
      if not(area >= 130 and area <= 170) :
        continue
      rects.append(rect)
      count += 1

    elif (top[0] > bottom[0]) and (top[1] > bottom[1]) :
      area = getArea(top, bottom)
      if not(area >= 130 and area <= 170) :
        continue
      rect = [bottom,top]
      rects.append(rect)
      count += 1

  return rects





In [17]:
def loadMNIST() :
  data= fetch_openml('mnist_784', version=1, return_X_y=False, )  
  X = data.data
  Y = data.target
  target_names = data.target_names
  featureNames =  data.feature_names
  return X, Y, target_names, featureNames
  
X, Y, target_names, featureNames= loadMNIST()

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


In [18]:
rects =  generateRandomRectangles(100)
rects

[[[7, 1], [13, 23]],
 [[15, 3], [22, 23]],
 [[8, 16], [26, 25]],
 [[2, 3], [10, 20]],
 [[4, 7], [13, 22]],
 [[1, 1], [14, 11]],
 [[5, 12], [16, 24]],
 [[13, 8], [26, 18]],
 [[13, 2], [26, 13]],
 [[9, 1], [16, 23]],
 [[0, 10], [24, 17]],
 [[1, 0], [20, 8]],
 [[3, 2], [10, 23]],
 [[1, 9], [14, 19]],
 [[5, 7], [18, 17]],
 [[3, 12], [15, 23]],
 [[7, 6], [26, 13]],
 [[8, 5], [20, 17]],
 [[15, 0], [23, 17]],
 [[6, 0], [22, 9]],
 [[13, 8], [22, 25]],
 [[6, 5], [16, 19]],
 [[6, 3], [23, 12]],
 [[13, 2], [26, 13]],
 [[12, 0], [18, 26]],
 [[2, 15], [22, 22]],
 [[2, 3], [14, 14]],
 [[1, 5], [22, 12]],
 [[10, 16], [26, 25]],
 [[5, 1], [12, 21]],
 [[9, 6], [26, 16]],
 [[4, 6], [22, 15]],
 [[1, 7], [24, 13]],
 [[16, 3], [26, 16]],
 [[0, 5], [15, 15]],
 [[8, 5], [16, 24]],
 [[8, 10], [21, 20]],
 [[13, 7], [22, 24]],
 [[15, 6], [25, 23]],
 [[2, 11], [17, 21]],
 [[7, 2], [19, 13]],
 [[8, 11], [22, 22]],
 [[5, 8], [17, 20]],
 [[5, 1], [25, 8]],
 [[15, 2], [22, 21]],
 [[1, 1], [18, 10]],
 [[13, 1], [22, 

In [21]:
def getBlackPixel(image):
  rows = image.shape[0]
  cols = image.shape[1]
  blackPoints = np.zeros((rows,cols))
  for j in range(cols) :
    if image[0][j] > 0 :
      if j > 0 :
        blackPoints[0][j] = blackPoints[0][j-1] + 1
      else:
        #  for [0,0]
        black[0][j] = 1

  for i in range(rows):
    if image[0][i] > 0:
      if i > 0 :
        blackPoints[i][0] = blackPoints[i-1][0] + 1
      else:
        blackPoints[i][0] = 1
  
  for i in range(1, rows):
    for j in range(1, cols):
      blackpoint_DIJ = 0
      if image[i][j] > 0:
        blackpoint_DIJ = 1
      blackPoints[i][j] = blackPoints[i][j-1]  + blackPoints[i-1][j]  -blackPoints[i-1][j-1]+ blackpoint_DIJ
  return blackPoints

In [ ]:
verticalRectFeatures = []
horizontalRectFeatures = []

index = 0
datapoint = np.zeros((len(X),200))
for img in tqdm(X):

  # if(index == 10000):
  #   break
  image = img.reshape(28,28)
  mat = getBlackPixel(image)
  index_y = 0
  for _rect in rects:
    top = _rect[0]
    bottom = _rect[1]

    topRight = [bottom[0], top[1]]
    bottomLeft = [top[0], bottom[1]]

    first = HalfArea(top, bottom, True)
    second = HalfArea(bottom, topRight, True)
    third = HalfArea(top, topRight, False)
    fourth = HalfArea(bottomLeft,bottom, False)
    # black(rectangle ABCD) = black(OTYD) - black(OTXB) - black(OZYC) + black(OZXA)
    # The last step is to compute the two feature (horizontal, vertical) values as differences:
    # vertical_feature_value(rectangle ABCD) = black(ABQR) - black(QRCD)
    # horizontal_feature_value(rectangle ABCD) = black(AMCN) - black(MBND)

    A1 = (mat[second[0], second[1]] - mat[topRight[0], topRight[1]] - 
           mat[first[0], first[1]] + mat[top[0], top[1]] )
    
    A2 = (mat[bottom[0], bottom[1]] - mat[second[0], second[1]] - 
           mat[bottomLeft[0], bottomLeft[1]] + mat[first[0], first[1]] )
    # 
    A3 = (mat[fourth[0], fourth[1]] - mat[third[0], third[1]] - 
           mat[bottomLeft[0], bottomLeft[1]] + mat[top[0], top[1]] )
    
    A4 = (mat[bottom[0], bottom[1]] - mat[topRight[0], topRight[1]] - 
           mat[fourth[0], fourth[1]] + mat[third[0], third[1]] )
    
    verticalFea = A1 - A2
    horizontalFea = A3 - A4

    datapoint[index] [index_y] = verticalFea
    datapoint[index] [index_y+1] = horizontalFea
    index_y+=2

  index += 1

In [23]:
X_train, X_test, y_train, y_test = train_test_split(datapoint, Y, test_size=0.33, random_state=42)

In [24]:
logisticClassfier = LogisticRegression(penalty='l2', tol =0.1, C = 1.0,n_jobs=-1)
logisticClassfier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.1, verbose=0,
                   warm_start=False)

In [25]:
print(f" train error with 200 Harr Features : {logisticClassfier.score(X_train, y_train)}")

 train error with 200 Harr Features : 0.9086140724946695


In [26]:
print(f"test Error with 200 Harr Features : {logisticClassfier.score(X_test, y_test)}")

test Error with 200 Harr Features : 0.905064935064935
